<a href="https://colab.research.google.com/github/Dkepffl/2022-1-ESAA/blob/main/HD_AI_Challenge/HD_FE_1016.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **HD 현대 AI Chalenge**
- 구글 드라이브 어마운트를 통해 데이터 로드
- 리더보드 최저 점수(26.8103796658) 코드


## **| 구글 드라이브 어마운트**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **| 필요한 라이브러리 로드**

In [ ]:
# import libraries
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import os

## **| 작업 폴더 변경 및 데이터 로드**

In [ ]:
os.chdir("/content/drive/MyDrive/HD현대 AI Challenge") # 각자 드라이브에 맞춰 변경
os.getcwd()

'/content/drive/MyDrive/HD현대 AI Challenge'

In [ ]:
# 결측치로 평균값으로 대체한 데이터셋 사용
train = pd.read_csv('train_na.csv').drop('Unnamed: 0', axis=1)
test = pd.read_csv('test_na.csv').drop('Unnamed: 0', axis=1)

In [ ]:
# 원본 데이터셋 별도 저장
train_proto = train.copy()
test_proto = test.copy()

## **| 데이터 전처리**

### **1. ATA 변수 처리**

In [ ]:
from datetime import datetime

# datetime 컬럼 처리
train['ATA'] = pd.to_datetime(train['ATA'])
test['ATA'] = pd.to_datetime(test['ATA'])

# datetime을 여러 파생 변수로 변환
for df in [train, test]:
  df['YEAR'] = df['ATA'].dt.year
  df['MONTH'] = df['ATA'].dt.month
  df['DAY'] = df['ATA'].dt.day
  df['HOUR'] = df['ATA'].dt.hour
  df['MINUTE'] = df['ATA'].dt.minute
  df['WEEKDAY'] = df['ATA'].dt.weekday

### **2. 파생 변수 추가**

#### **월 평균 Oil 가격 컬럼 생성**

In [ ]:
# 파생 변수 생성에 필요한 필요한 컬럼 추출
oil_price_train = train[['YEAR','MONTH','DAY', 'WTI', 'BRENT', 'DUBAI']]
oil_price_test = test[['YEAR','MONTH','DAY', 'WTI', 'BRENT', 'DUBAI']]

In [ ]:
# 연도별 월 평균 oil 가격 컬럼 생성
## train 데이터셋
oil_price_train['YYMM'] = oil_price_train['YEAR'].apply(str)+oil_price_train['MONTH'].apply(str)
oil_mean_train = oil_price_train.groupby('YYMM').mean()
oil_mean_train = oil_mean_train.sort_values(by=['YEAR', 'MONTH']).reset_index(drop=True)

## test 데이터셋
oil_price_test['YYMM']  = oil_price_test['YEAR'].apply(str)+oil_price_test['MONTH'].apply(str)
oil_mean_test = oil_price_test.groupby('YYMM').mean()
oil_mean_test = oil_mean_test.sort_values(by=['YEAR', 'MONTH']).reset_index(drop=True)

<ipython-input-8-48886d8fd197>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oil_price_train['YYMM'] = oil_price_train['YEAR'].apply(str)+oil_price_train['MONTH'].apply(str)
<ipython-input-8-48886d8fd197>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oil_price_test['YYMM']  = oil_price_test['YEAR'].apply(str)+oil_price_test['MONTH'].apply(str)


In [ ]:
oil_mean_train

,YEAR,MONTH,DAY,WTI,BRENT,DUBAI
0,2014.0,9.0,12.484632,92.817555,98.252302,96.345247
1,2014.0,10.0,15.872201,82.912920,86.770308,85.299907
2,2014.0,11.0,15.553971,73.459277,77.102057,74.497760
3,2014.0,12.0,14.689000,58.068470,61.990670,58.831420
4,2015.0,1.0,16.090600,47.517895,50.159562,46.270183
...,...,...,...,...,...,...
97,2022.0,10.0,16.041744,86.967848,93.562752,90.968680
98,2022.0,11.0,17.003387,82.375961,89.106047,84.504075
99,2022.0,12.0,15.321127,76.345763,81.191680,77.214503
100,2023.0,1.0,16.178439,78.163917,83.901876,80.846013


In [ ]:
# WTI, BRENT, DUBAI 컬럼 추출
# np.shape(oil_price_lst_train) : (3, 102)
oil_price_lst_train = [oil_mean_train['WTI'].values, oil_mean_train['DUBAI'].values, oil_mean_train['BRENT'].values]
oil_price_lst_test = [oil_mean_test['WTI'].values, oil_mean_test['DUBAI'].values, oil_mean_test['BRENT'].values]

In [ ]:
def get_lst_idx(year, month):
    base = 3
    if year==2014:
        if month==9:return 0
        else: return month - 10
    else: return base + (year - 2015)*12 + (month - 1)

In [ ]:
from tqdm import tqdm

oil_befs_train = [[],[],[]]
oil_befs_test = [[],[],[]]

# 지난달 oil가격 추가
for d in tqdm(train.iterrows()):
    idx_train = int(get_lst_idx(d[1]['YEAR'], d[1]['MONTH']))
    oil_befs_train[0].append(oil_price_lst_train[0][idx_train])
    oil_befs_train[1].append(oil_price_lst_train[1][idx_train])
    oil_befs_train[2].append(oil_price_lst_train[2][idx_train])

for d in tqdm(test.iterrows()):
    idx_test = int(get_lst_idx(d[1]['YEAR'], d[1]['MONTH']))
    oil_befs_test[0].append(oil_price_lst_test[0][idx_test])
    oil_befs_test[1].append(oil_price_lst_test[1][idx_test])
    oil_befs_test[2].append(oil_price_lst_test[2][idx_test])

367441it [00:32, 11464.93it/s]
244989it [00:16, 15224.16it/s]


In [ ]:
train['WTI_BEF'] = oil_befs_train[0]
train['BRENT_BEF'] = oil_befs_train[1]
train['DUBAI_BEF'] = oil_befs_train[2]

In [ ]:
test['WTI_BEF'] = oil_befs_test[0]
test['BRENT_BEF'] = oil_befs_test[1]
test['DUBAI_BEF'] = oil_befs_test[2]

#### **ATA_PO 별 DIST 통계량**

In [ ]:
DIST_dist_train=train.groupby('ARI_PO')['DIST'].agg(['mean','std']).reset_index()
DIST_dist_test=train.groupby('ARI_PO')['DIST'].agg(['mean','std']).reset_index()

In [ ]:
DIST_dist_train.loc[DIST_dist_train['ARI_PO']=='EKP8','mean'].values[0]

14.43483822177967

In [ ]:
port_list1=train['ARI_PO'].unique().tolist()
port_list2=test['ARI_PO'].unique().tolist()

for port in port_list1:
  train.loc[train['ARI_PO']==port,'DIST_MEAN'] = DIST_dist_train.loc[DIST_dist_train['ARI_PO']==port,'mean'].values[0]
  train.loc[train['ARI_PO']==port,'DIST_STD'] = DIST_dist_train.loc[DIST_dist_train['ARI_PO']==port,'std'].values[0]

In [ ]:
for port in port_list2:
  test.loc[test['ARI_PO']==port,'DIST_MEAN'] = DIST_dist_test.loc[DIST_dist_test['ARI_PO']==port,'mean'].values[0]
  test.loc[test['ARI_PO']==port,'DIST_STD'] = DIST_dist_test.loc[DIST_dist_test['ARI_PO']==port,'std'].values[0]

### **3. 카테고리 변수 라벨 인코딩**

In [ ]:
from sklearn.preprocessing import LabelEncoder
import bisect

categorical_features = ['ARI_CO','ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER','FLAG']
encoders = {}

for feature in tqdm(categorical_features, desc="Encoding features"):
  le = LabelEncoder()
  train[feature] = le.fit_transform(train[feature].astype(str))
  le_classes_set = set(le.classes_)
  test[feature] = test[feature].map(lambda s: '-1' if s not in le_classes_set else s)
  le_classes = le.classes_.tolist()
  bisect.insort_left(le_classes, '-1')
  le.classes_ = np.array(le_classes)
  test[feature] = le.transform(test[feature].astype(str))
  encoders[feature] = le

Encoding features: 100%|██████████| 6/6 [00:02<00:00,  3.00it/s]


### **4. 미사용 컬럼 제거**

In [ ]:
train.drop(columns=['ID', 'SHIPMANAGER', 'FLAG', 'U_WIND', 'V_WIND', 'AIR_TEMPERATURE', 'BUILT'], inplace=True)
test.drop(columns=['ID', 'SHIPMANAGER', 'FLAG', 'U_WIND', 'V_WIND', 'AIR_TEMPERATURE', 'BUILT'], inplace=True)

In [ ]:
train = train.fillna(-1)
test = test.fillna(-1)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367441 entries, 0 to 367440
Data columns (total 30 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   ARI_CO              367441 non-null  int64         
 1   ARI_PO              367441 non-null  int64         
 2   SHIP_TYPE_CATEGORY  367441 non-null  int64         
 3   DIST                367441 non-null  float64       
 4   ATA                 367441 non-null  datetime64[ns]
 5   BREADTH             367441 non-null  float64       
 6   DEADWEIGHT          367441 non-null  int64         
 7   DEPTH               367441 non-null  float64       
 8   DRAUGHT             367441 non-null  float64       
 9   GT                  367441 non-null  int64         
 10  LENGTH              367441 non-null  float64       
 11  BN                  367441 non-null  float64       
 12  ATA_LT              367441 non-null  int64         
 13  DUBAI               367441 no

### **5. 스케일링**

In [ ]:
from sklearn.preprocessing import StandardScaler

scl = StandardScaler()

train_scld = scl.fit_transform(train.drop(['ATA', 'CI_HOUR'], axis=1))
train_scld = pd.DataFrame(train_scld, columns=train.drop(['ATA', 'CI_HOUR'], axis=1).columns)
train_scld['ATA']=train['ATA']

test_scld = scl.transform(test.drop('ATA', axis=1))
test_scld = pd.DataFrame(test_scld, columns=test.drop('ATA', axis=1).columns)
test_scld['ATA']=test['ATA']

## **| 모델 학습**

### **학습/검증 데이터셋 생성**

In [ ]:
from sklearn.model_selection import train_test_split
X = train_scld
Y = train["CI_HOUR"]
x_train, x_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2, random_state=42)

### **Catboost 모델링**

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 2.9 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostRegressor

cbc = CatBoostRegressor(verbose=1000, iterations=20000, objective='MAE', use_best_model=True)
cbc.fit(x_train, y_train, eval_set=(x_valid, y_valid))

0:	learn: 60.7084188	test: 59.8731411	best: 59.8731411 (0)	total: 164ms	remaining: 54m 31s
1000:	learn: 43.9931427	test: 43.3483073	best: 43.3483073 (1000)	total: 1m 41s	remaining: 32m 14s
2000:	learn: 41.7270099	test: 41.1672639	best: 41.1672639 (2000)	total: 3m 10s	remaining: 28m 29s
3000:	learn: 39.4055488	test: 38.9719776	best: 38.9719776 (3000)	total: 4m 37s	remaining: 26m 14s
4000:	learn: 37.1721706	test: 36.8347407	best: 36.8347407 (4000)	total: 6m 3s	remaining: 24m 15s
5000:	learn: 35.2894937	test: 35.0311512	best: 35.0311512 (5000)	total: 7m 30s	remaining: 22m 30s
6000:	learn: 34.1796384	test: 33.9892840	best: 33.9892840 (6000)	total: 8m 59s	remaining: 20m 58s
7000:	learn: 33.3487169	test: 33.1789621	best: 33.1789621 (7000)	total: 10m 25s	remaining: 19m 21s
8000:	learn: 32.7709642	test: 32.6301644	best: 32.6301644 (8000)	total: 11m 49s	remaining: 17m 44s
9000:	learn: 32.1027528	test: 31.9902252	best: 31.9902252 (9000)	total: 13m 15s	remaining: 16m 11s
10000:	learn: 31.3615273	

In [ ]:
from sklearn.metrics import mean_absolute_error

pred = cbc.predict(x_valid)
mean_absolute_error(pred, y_valid)

27.51203585523139

In [ ]:
# test 데이터셋에 대한 예측
final = cbc.predict(test)

## **| 제출 파일 생성 및 저장**

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/HD현대 AI Challenge/Data/sample_submission.csv')
submit['CI_HOUR'] = final

In [ ]:
submit.to_csv('sub_FE1016.csv', index=False) # 파일 이름 수정